Note: These exercise notebooks are meant to help you to get started with the exercises. They are meant to be incomplete, so you will have to add some steps yourself (often indicated by the "..." in the code). The number of steps that have to be added by the participants progressively increases the further you get with the exercises. If you can't figure it out yourself, no worries, have a look at the [solution](../solutions) folder or ask one of the teachers.

# Optical flow and extrapolation

In this notebook we show how to apply different optical flow methods to estimate the advection field. This is followed by applying semi-Lagrangian extrapolation to produce a simple advection-based nowcast.

First, we load the data to be used in the notebook.

In [ ]:
# Add here the function that you have used in block_02
...

In [ ]:
# Load your dataset (e.g. the radar dataset)
...

## Optical flow

First we estimate the advection field from a radar image sequence. This is the first step of all nowcasting methods currently implemented in pysteps. Here it will be done by using three different methods: Lucas-Kanade [1], VET [2] and DARTS [3]. For a list of available methods, see the documentation of the [motion](https://pysteps.readthedocs.io/en/stable/pysteps_reference/motion.html) module.

The Lucas-Kanade method itself is a sparse feature tracker rather than an optical flow method. In pysteps we have modified its OpenCV implementation [1] to produce dense advection fields. In the first stage of the method, a set of feature points corresponding to large image gradients are detected from the input images. This is followed by estimation of motion vectors for the above features. In the third stage, spatial interpolation is applied to the motion vectors to produce a smooth field covering the whole domain.

On the other hand, VET estimates the advection field directly over the whole domain using a variational approach. For this reason, it is computationally more expensive than Lucas-Kanade, but generally produces smoother fields.

In DARTS, the motion field solving the advection equation is determined in the Fourier domain. The main advantage of this approach is that it conveniently allows determining the motion in different spatial scales by choosing the number of Fourier frequencies for the inputs and outputs. As a result, it can determine the global advection field but also act as a local feature tracker.

The classical optical flow methods take only two input images. It is advantageous to estimate the advection field over a longer time series to apply temporal smoothing and thus make the estimation more stable. This option is included in our Lucas-Kanade implementation and DARTS, for which this property follows from the definition of the method.

We use [visualization.motionfields.quiver](https://pysteps.readthedocs.io/en/stable/generated/pysteps.visualization.motionfields.quiver.html#pysteps.visualization.motionfields.quiver) for plotting the motion fields on top of the map and the precipitation field. The most relevant parameters here are `step` and `scale` that determine the distance between the motion arrows and their length. For a detailed description of the scale parameter, see the documentation of [matplotlib.axes.Axes.quiver](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.quiver.html).

In [ ]:
import time
from matplotlib import pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings("ignore")

from pysteps import motion
from pysteps.utils import transformation
from pysteps.visualization import plot_precip_field, quiver

methods = ["LK", "VET", "DARTS"]
fig = plt.figure(figsize=(6, 10), layout="constrained")

# Set the time index for the issue time (the forecast, but also the motion fields
# will be calculated for this time step)
index = 131

# When computing the optical flow for precipitation intensity,
# transforming the precipitation rates (mm/h)
# to dBR via the logarithmic transform shown in the previous exercise generally
# improves the reliability of the estimation. However, if we want to emphasize
# estimation of the motion of cells containing the highest precipitation rates,
# it could be a good idea to apply optical flow directly without the
# transformation.

# Handling of NaN values has been explicitly implemented in Lucas-Kanade and VET,
# but not in DARTS. For this reason, we set all non-finite values to the minimum
# value before applying the optical flow.
precip_finite = precip[:index+1]
precip_finite[~np.isfinite(precip_finite)] = np.nanmin(precip)

for i, method in enumerate(methods):
    # Use get_method to get a callable function for the chosen method
    oflow = motion.get_method(method)

    starttime = time.time()

    if method == "LK":
        # Use 4 most recent precipitation fields (60 minutes) for Lucas-Kanade
        # Disable printing with verbose=False
        advection = oflow(precip_finite[-4:], verbose=False)
    elif method == "VET":
        # VET takes two or three input fields
        advection = oflow(precip_finite[-3:], verbose=False)
    elif method == "DARTS":
        # Use 8 most recent precipitation fields (two hours) for DARTS that
        # performs the best when supplied with a long time series
        advection = oflow(precip_finite[-8:], verbose=False)

    print(f"Optical flow computation with {method} took {time.time() - starttime:.03f} seconds")

    ax = fig.add_subplot(311 + i); ax.axis("off")

    plot_precip_field(..., geodata=..., title=method, ax=ax)
    quiver(advection, geodata=..., step=15, quiver_kwargs={"scale": 200})

plt.show()

As seen from above, Lucas-Kanade and VET produce smooth motion fields that look physically credible, albeit quite different in this example. The former is the fastest, so we recommend using it as the default choice. On the other hand, DARTS produces artificial rotation patterns particularly in regions of no precipitation. This can be improved by adding more Fourier wavenumbers (the N_x, N_y, M_x and M_y parameters), but this will significantly increase computation time.

In [ ]:
# Let's add some transformations, to see if this helps with the advection field
# derivation.

from pysteps.utils import transformation, conversion

# First transform the precip from mm to mm/h
precip_mmh, metadata_mmh = conversion.to_rainrate(precip_finite, metadata)

# Then, log-transform the data to dBR with threshold of 0.1 mm/h and fill value of
# -15 dBR
precip_dbr, metadata_dbr = transformation.dB_transform(
    precip_mmh,
    metadata_mmh,
    threshold=0.1,
    zerovalue=-15.0
)

|# Now, follow the same steps again to plot it.
...

## Extrapolation

Using the estimated advection field, we can now apply the semi-Lagrangian extrapolation. In pysteps we have implemented the method described in [4] in [extrapolation.semilagrangian.extrapolate](https://pysteps.readthedocs.io/en/stable/generated/pysteps.extrapolation.semilagrangian.extrapolate.html#pysteps.extrapolation.semilagrangian.extrapolate).

In [ ]:
from pysteps.extrapolation.semilagrangian import extrapolate

# Extrapolate the last observed precipitation field to 12 time steps (one hour)
# forward in 15-minute time steps. The composite contains NaN values outside the
# radar domain, so we need to use the allow_nonfinite_values option.
oflow = ...
advection = oflow(precip_dbr[-4:], verbose=False)
precip_extrap = extrapolate(..., ..., index, allow_nonfinite_values=True)

fig = plt.figure(figsize=(6, 12), layout="constrained")

# Note that even if we are interested in a
# small number of future time steps, the extrapolation should be done by using
# as large number of time steps (and iterations, see the n_iter argument) as
# possible to achieve the best accuracy. This is because we are integrating the
# movement of precipitation along a nonlinear advection field.
for i in range(4):
    ax = fig.add_subplot(4, 1, 1 + i)
    plot_precip_field(...)
    ax.set_title(f"{(i + 1) * 15} minutes")

plt.show()

The above produces the simplest possible nowcast, also called the *Lagrangian persistence*. Equivalently, this can be done by calling `nowcasts.get_method("extrapolation")`.

In the above example we used a fixed number of time steps, whose length is the same as in the input time series. However, in some cases we need to use a different time step for the outputs. One example is computation of accumulated precipitation, where having a high temporal resolution is essential. This can be done by supplying a list of time steps (relative to the input time step) as the third argument of `extrapolate`. In this example, we compute the extrapolated time series to the next 15 minutes with 1-minute time step.

In [ ]:
# here the time step is scaled so that 1 corresponds to the time step between
# successive inputs
timesteps = np.linspace(0., 1.0, 15)
print(timesteps)
precip_extrap = extrapolate(precip[index], advection, timesteps, allow_nonfinite_values=True)
precip_extrap.shape

For a description of additional arguments for `extrapolate`, see the [pysteps documentation](https://pysteps.readthedocs.io/en/stable/generated/pysteps.extrapolation.semilagrangian.extrapolate.html#pysteps.extrapolation.semilagrangian.extrapolate).

## References


[1] J.-Y. Bouguet. Pyramidal implementation of the affine lucas kanade feature tracker description of the algorithm. Intel Corporation, 5, 2001

[2] S. Laroche and I. Zawadzki. Retrievals of Horizontal Winds from Single-Doppler Clear-Air Data by Methods of Cross Correlation and Variational Analysis. Journal of Atmospheric and Oceanic Technology, 12(4): 721-738, 1995, doi: 10.1175/1520-0426(1995)012<0721:ROHWFS>2.0.CO;2

[3] E. Ruzanski, V. Chandrasekar, and Y. Wang. The CASA nowcasting system. Journal of Atmospheric and Oceanic Technology, 28(5):640–655, 2011. doi:10.1175/2011JTECHA1496.1

[4] U. Germann and I. Zawadzki. Scale-dependence of the predictability of precipitation from continental radar images. Part I: description of the methodology. Monthly Weather Review, 130(12):2859–2873, 2002. doi:10.1175/1520-0493(2002)130<2859:SDOTPO>2.0.CO;2.